<a href="https://colab.research.google.com/github/JingRu-hi/ml2022/blob/main/ml2022hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# If you want to access the version you have already modified, click "Edit"
# If you want to access the original sample code, click "...", then click "Copy & Edit Notebook"

In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# packages

In [ ]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
# import torchvision
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
from sklearn.model_selection import KFold
# This is for the progress bar.
from tqdm.auto import tqdm
import random
import sklearn.model_selection


import torchvision.models as models

In [ ]:
_exp_name = "sample"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
myseed = 666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

## **Transforms**
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [ ]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    transforms.RandomGrayscale(),
    transforms.ColorJitter(),
    transforms.RandomPerspective(),
    transforms.RandomRotation(degrees=30), #(-30,30)
    transforms.RandomHorizontalFlip(),
    # You need to add some transforms here.
    transforms.ToTensor(),
])


## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [ ]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        # im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label



# network

In [ ]:
resnet18 = models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
print(resnet18)

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            # nn.Dropout(),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            # nn.Dropout(),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            # nn.Dropout(),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            # nn.Dropout(),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            # nn.Dropout(),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [ ]:
#! wget https://www.dropbox.com/s/6l2vcvxl54b0b6w/food11.zip
! wget -O food11.zip "https://github.com/virginiakm1988/ML2022-Spring/blob/main/HW03/food11.zip?raw=true"

--2022-03-30 16:33:05--  https://github.com/virginiakm1988/ML2022-Spring/blob/main/HW03/food11.zip?raw=true
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/virginiakm1988/ML2022-Spring/raw/main/HW03/food11.zip [following]
--2022-03-30 16:33:05--  https://github.com/virginiakm1988/ML2022-Spring/raw/main/HW03/food11.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/virginiakm1988/ML2022-Spring/main/HW03/food11.zip [following]
--2022-03-30 16:33:05--  https://media.githubusercontent.com/media/virginiakm1988/ML2022-Spring/main/HW03/food11.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185

In [ ]:
! unzip food11.zip

# cv fn

In [ ]:
# CV
def cv_train(train_loader, valid_loader,fold,
             n_epochs = 1, patience = 5
             ):
  # "cuda" only when GPUs are available.
  device = "cuda" if torch.cuda.is_available() else "cpu"

  # The number of training epochs and patience.
  n_epochs = n_epochs
  patience = patience # If no improvement in 'patience' epochs, early stop

  # Initialize a model, and put it on the device specified.
  # model = Classifier().to(device)
  model = models.resnet18(pretrained=True).to(device)
  # model.load_state_dict(torch.load(f"sample_best.ckpt")) ######

  # For the classification task, we use cross-entropy as the measurement of performance.
  criterion = nn.CrossEntropyLoss()
  # Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5) 

  # Initialize trackers, these are not parameters and should not be changed
  stale = 0
  best_acc = 0

  for epoch in range(n_epochs):

      # ---------- Training ----------
      # Make sure the model is in train mode before training.
      model.train()
      # These are used to record information in training.
      train_loss = []
      train_accs = []

      for batch in tqdm(train_loader):

          # A batch consists of image data and corresponding labels.
          imgs, labels = batch
          #imgs = imgs.half()
          #print(imgs.shape,labels.shape)

          # Forward the data. (Make sure data and model are on the same device.)
          logits = model(imgs.to(device))

          # Calculate the cross-entropy loss.
          # We don't need to apply softmax before computing cross-entropy as it is done automatically.
          loss = criterion(logits, labels.to(device))

          # Gradients stored in the parameters in the previous step should be cleared out first.
          optimizer.zero_grad()

          # Compute the gradients for parameters.
          loss.backward()

          # Clip the gradient norms for stable training.
          grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

          # Update the parameters with computed gradients.
          optimizer.step()

          # Compute the accuracy for current batch.
          acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

          # Record the loss and accuracy.
          train_loss.append(loss.item())
          train_accs.append(acc)
          
      train_loss = sum(train_loss) / len(train_loss)
      train_acc = sum(train_accs) / len(train_accs)

      # Print the information.
      print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

      # ---------- Validation ----------
      # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
      model.eval()

      # These are used to record information in validation.
      valid_loss = []
      valid_accs = []

      # Iterate the validation set by batches.
      for batch in tqdm(valid_loader):

          # A batch consists of image data and corresponding labels.
          imgs, labels = batch
          #imgs = imgs.half()

          # We don't need gradient in validation.
          # Using torch.no_grad() accelerates the forward process.
          with torch.no_grad():
              logits = model(imgs.to(device))

          # We can still compute the loss (but not the gradient).
          loss = criterion(logits, labels.to(device))

          # Compute the accuracy for current batch.
          acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

          # Record the loss and accuracy.
          valid_loss.append(loss.item())
          valid_accs.append(acc)
          #break

      # The average loss and accuracy for entire validation set is the average of the recorded values.
      valid_loss = sum(valid_loss) / len(valid_loss)
      valid_acc = sum(valid_accs) / len(valid_accs)

      # Print the information.
      print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


      # update logs
      if valid_acc > best_acc:
          with open(f"./{_exp_name}{fold}_log.txt","a"):
              print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
      else:
          with open(f"./{_exp_name}{fold}_log.txt","a"):
              print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


      # save models
      if valid_acc > best_acc:
          print(f"Best model found at epoch {epoch}, saving model")
          torch.save(model.state_dict(), f"{_exp_name}{fold}_best.ckpt") # only save best to prevent output memory exceed error
          best_acc = valid_acc
          stale = 0
      else:
          stale += 1
          if stale > patience:
              print(f"No improvment {patience} consecutive epochs, early stopping")
              break

In [ ]:
# all in
class FoodDataset(Dataset):

    def __init__(self,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        # self.path = path
        # self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        self.files_fromT = [os.path.join('training',x) for x in os.listdir('training') if x.endswith(".jpg")]
        self.files_fromV = [os.path.join('validation',x) for x in os.listdir('validation') if x.endswith(".jpg")]
        # self.files =
        if files != None:
            self.files = files
        # print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        # im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

In [ ]:
# CV
batch_size = 256
_dataset_dir = "./food11"
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set_t = FoodDataset(os.path.join(_dataset_dir,"training"), tfm=train_tfm)
valid_set_t = FoodDataset(os.path.join(_dataset_dir,"validation"), tfm=train_tfm)
cvt_set = ConcatDataset([train_set_t,valid_set_t])

train_set_v = FoodDataset(os.path.join(_dataset_dir,"training"), tfm=test_tfm)
valid_set_v = FoodDataset(os.path.join(_dataset_dir,"validation"), tfm=test_tfm)
cvv_set = ConcatDataset([train_set_v,valid_set_v])

# cv_loader = DataLoader(cv_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

In [ ]:
# CV
batch_size = 256
_dataset_dir = "./food11"
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = FoodDataset(os.path.join(_dataset_dir,"training"))#, tfm=train_tfm)
# train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"))#, tfm=test_tfm)
# valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
cv_set = ConcatDataset([train_set,valid_set])
# cv_loader = DataLoader(cv_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

One ./food11/training sample ./food11/training/0_0.jpg
One ./food11/validation sample ./food11/validation/0_0.jpg
One ./food11/test sample ./food11/test/0001.jpg


# 開train

In [ ]:
class WrapperDataset:
    def __init__(self, dataset, transform=None, target_transform=None):
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        image, label = self.dataset[index]
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.dataset)

In [ ]:
cvt_set[0][1]

0

In [ ]:
kf = KFold(n_splits=4)
n_epochs = 10

for fold,(train_idx,val_idx) in enumerate(kf.split(cvt_set)):
  print('------------fold no---------{}----------------------'.format(fold))
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
  val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
  train_loader = DataLoader(
        WrapperDataset(cvt_set, transform=train_tfm),
        batch_size=batch_size, sampler=train_subsampler)
  valid_loader = DataLoader(
        WrapperDataset(cvt_set, transform=test_tfm),
        batch_size=batch_size, sampler=val_subsampler)
  
  # model.apply(reset_weights)
  # parameters: train_loader, valid_loader,fold, n_epochs = 1, patience = 300
  device = "cuda" if torch.cuda.is_available() else "cpu"

  # The number of training epochs and patience.
  n_epochs = n_epochs
  patience = 5 # If no improvement in 'patience' epochs, early stop

  # Initialize a model, and put it on the device specified.
  # model = Classifier().to(device)
  model = models.resnet18(pretrained=False).to(device)
  # model.load_state_dict(torch.load(f"sample_best.ckpt")) ######

  # For the classification task, we use cross-entropy as the measurement of performance.
  criterion = nn.CrossEntropyLoss()
  # Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5) 

  # Initialize trackers, these are not parameters and should not be changed
  stale = 0
  best_acc = 0

  for epoch in range(n_epochs):

      # ---------- Training ----------
      # Make sure the model is in train mode before training.
      model.train()
      # These are used to record information in training.
      train_loss = []
      train_accs = []

      for batch in tqdm(train_loader):

          # A batch consists of image data and corresponding labels.
          imgs, labels = batch
          #imgs = imgs.half()
          #print(imgs.shape,labels.shape)

          # Forward the data. (Make sure data and model are on the same device.)
          logits = model(imgs.to(device))

          # Calculate the cross-entropy loss.
          # We don't need to apply softmax before computing cross-entropy as it is done automatically.
          loss = criterion(logits, labels.to(device))

          # Gradients stored in the parameters in the previous step should be cleared out first.
          optimizer.zero_grad()

          # Compute the gradients for parameters.
          loss.backward()

          # Clip the gradient norms for stable training.
          grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

          # Update the parameters with computed gradients.
          optimizer.step()

          # Compute the accuracy for current batch.
          acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

          # Record the loss and accuracy.
          train_loss.append(loss.item())
          train_accs.append(acc)
          
      train_loss = sum(train_loss) / len(train_loss)
      train_acc = sum(train_accs) / len(train_accs)

      # Print the information.
      print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

      # ---------- Validation ----------
      # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
      model.eval()

      # These are used to record information in validation.
      valid_loss = []
      valid_accs = []

      # Iterate the validation set by batches.
      for batch in tqdm(valid_loader):

          # A batch consists of image data and corresponding labels.
          imgs, labels = batch
          #imgs = imgs.half()

          # We don't need gradient in validation.
          # Using torch.no_grad() accelerates the forward process.
          with torch.no_grad():
              logits = model(imgs.to(device))

          # We can still compute the loss (but not the gradient).
          loss = criterion(logits, labels.to(device))

          # Compute the accuracy for current batch.
          acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

          # Record the loss and accuracy.
          valid_loss.append(loss.item())
          valid_accs.append(acc)
          #break

      # The average loss and accuracy for entire validation set is the average of the recorded values.
      valid_loss = sum(valid_loss) / len(valid_loss)
      valid_acc = sum(valid_accs) / len(valid_accs)

      # Print the information.
      print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


      # update logs
      if valid_acc > best_acc:
          with open(f"./{_exp_name}{fold}_log.txt","a"):
              print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
      else:
          with open(f"./{_exp_name}{fold}_log.txt","a"):
              print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


      # save models
      if valid_acc > best_acc:
          print(f"Best model found at epoch {epoch}, saving model")
          torch.save(model.state_dict(), f"{_exp_name}{fold}_best.ckpt") # only save best to prevent output memory exceed error
          best_acc = valid_acc
          stale = 0
      else:
          stale += 1
          if stale > patience:
              print(f"No improvment {patience} consecutive epochs, early stopping")
              break
# for epoch in range(1, epochs + 1):
#   train(fold, model, device, trainloader, optimizer, epoch)
#   val(fold,model, device, valloader)

------------fold no---------0----------------------


  0%|          | 0/39 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
kf = KFold(n_splits=4)
n_epochs = 10

for fold,(train_idx,val_idx) in enumerate(kf.split(cvt_set)):
  print('------------fold no---------{}----------------------'.format(fold))
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
  val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
  train_loader = DataLoader(cvt_set, batch_size=batch_size, sampler=train_subsampler)
  valid_loader = DataLoader(cvv_set, batch_size=batch_size, sampler=val_subsampler)
 
  # model.apply(reset_weights)
  # parameters: train_loader, valid_loader,fold, n_epochs = 1, patience = 300
  cv_train(train_loader, valid_loader, fold,  n_epochs = n_epochs)
  # for epoch in range(1, epochs + 1):
  #   train(fold, model, device, trainloader, optimizer, epoch)
  #   val(fold,model, device, valloader)

------------fold no---------0----------------------


  0%|          | 0/39 [00:00<?, ?it/s]

[ Train | 001/010 ] loss = 2.41510, acc = 0.59986


  0%|          | 0/13 [00:00<?, ?it/s]

[ Valid | 001/010 ] loss = 3.63572, acc = 0.19220
[ Valid | 001/010 ] loss = 3.63572, acc = 0.19220 -> best
Best model found at epoch 0, saving model


...

[ Train | 001/010 ] loss = 2.40542, acc = 0.59614


  0%|          | 0/13 [00:00<?, ?it/s]

[ Valid | 001/010 ] loss = 2.93026, acc = 0.23315
[ Valid | 001/010 ] loss = 2.93026, acc = 0.23315 -> best
Best model found at epoch 0, saving model


  0%|          | 0/39 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Testing and generate prediction CSV

In [ ]:
prediction = []
clf0, clf1, clf2, clf3 = Classifier().to(device), Classifier().to(device), Classifier().to(device), Classifier().to(device)
clf0.load_state_dict(torch.load(f"{_exp_name}0_best.ckpt"))
clf0.eval()
clf1.load_state_dict(torch.load(f"{_exp_name}1_best.ckpt"))
clf1.eval()
clf2.load_state_dict(torch.load(f"{_exp_name}2_best.ckpt"))
clf2.eval()
clf3.load_state_dict(torch.load(f"{_exp_name}3_best.ckpt"))
clf3.eval()

with torch.no_grad():
    for batch, (data,_) in enumerate(test_loader):
      voting_arr = [[] for _ in range(4)]
      test_pred0 = clf0(data.to(device)) # probability
      voting_arr[0] = np.argmax(test_pred0.cpu().data.numpy(), axis=1)
      test_pred1 = clf1(data.to(device)) # probability
      voting_arr[1] = np.argmax(test_pred1.cpu().data.numpy(), axis=1)
      test_pred2 = clf2(data.to(device)) # probability
      voting_arr[2] = np.argmax(test_pred2.cpu().data.numpy(), axis=1)
      test_pred3 = clf3(data.to(device)) # probability
      voting_arr[3] = np.argmax(test_pred3.cpu().data.numpy(), axis=1)
      # get mode
      values, counts = np.unique(voting_arr, return_counts=True,axis=0)
      idx = np.argmax(counts)
      mode = values[idx]
      prediction += mode.tolist()
      # break


In [ ]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)